# **Cassava Neural Net prediction**
2021/01/11 written by T.Yonezu

In [10]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader, Dataset

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import glob 
import os
from tqdm import tqdm

from cassava_dataset import *
from myvgg import *

import warnings
warnings.simplefilter('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
input_dir = os.path.join('..',"..", 'input', 'cassava-leaf-disease-classification')

In [12]:
x = np.atleast_2d(glob.glob(os.path.join(input_dir, 'test_images',"*.jpg")))
x = x.T
x = pd.DataFrame(columns=["image_path"], data=x)
tmp = x["image_path"].str.split(os.path.sep,expand=True)
x["image_id"] = tmp[len(tmp.columns)-1]
x["label"] = np.nan

test_dict = dict( zip(x["image_path"],x["label"]) )

In [13]:
size = (224,224)
mean = [0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

transform = ImageTransform(size,mean,std)
test_data = CassavaDataset(test_dict,transform=transform)
test_data = DataLoader(test_data,batch_size=30)

In [14]:
EPOCH_NUM = 3

OUT_DIR = os.path.join("..","..","input","cassava-models")
MODEL_NAME = "VGG_cassava_trained_%dEpoch.mdl"
PATH = os.path.join(OUT_DIR,MODEL_NAME%EPOCH_NUM)

In [15]:
model = VGG11(num_classes=5)
model.load_state_dict(torch.load(PATH))
model.eval()
model = model.cuda()

pred_class = np.array([])

# in your training loop:
for batch in tqdm(test_data):
    
    X = batch[0].cuda()
    y = batch[1].cuda()

    pred = model(X)
    pred = pred.cpu()
    pred = torch.argmax(pred,axis=1)
    pred_class = np.append(pred_class,pred.numpy())

#submission
sub_df = x
sub_df["label"] = pred_class
sub_df = sub_df[["image_id","label"]]
sub_df.to_csv("submission.csv",index=False)

sub_df

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 52.85it/s]


,image_id,label
0,2216849948.jpg,3.0
